In [1]:
#pip install googletrans==3.1.0a0

In [2]:
from googletrans import Translator,LANGUAGES

# Google Translator

In [3]:
def translate(text):
    translator=Translator()
    translated=translator.translate(text=text,dest="english")
    return translated.text

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


# NLTK Stopwords

In [5]:
import re
import nltk
from nltk.util import pr
stemmer=nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
#nltk.download('stopwords')
stopword=set(stopwords.words("english"))
print(stopword)

{'my', 'didn', "aren't", 'had', 'into', 'from', 'shouldn', 'by', 'doesn', 'over', 'under', "hasn't", 'here', 'than', 'her', "wasn't", 're', 'have', 'o', 'couldn', 'hasn', "won't", 'before', "it's", 'after', 'they', 'yours', 'for', 'themselves', 'ours', 'that', "needn't", 'been', 'in', 'shan', 'down', 'now', "mustn't", 'isn', 'no', "shan't", 'same', "couldn't", 'hadn', 'what', 'about', 'y', 'while', 'how', 'both', 'doing', 'because', "you'll", 'out', 'each', 'did', 'will', 'just', 'and', 'why', 'needn', 'ma', 'is', 'most', 'i', 'so', "didn't", 'its', "wouldn't", 'mightn', 'can', 'during', 'wasn', 'you', 'it', 'has', 'other', 'do', 'once', 'weren', 'itself', 'when', 'more', 'was', 'having', 'but', 'an', "should've", 'wouldn', 'with', 'until', 'our', 'which', 'whom', 'very', 'through', 'being', "you're", 'she', 'does', 'yourself', 've', "don't", 'to', "shouldn't", 'are', 'we', 'up', "that'll", 'off', 'm', 'these', 'the', 'were', 'further', 'aren', "isn't", 'am', 's', 'him', 'ain', 'at', '

# Twitter Dataset

In [6]:
df=pd.read_csv("twitter_data.csv")
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


# Labelling of tweets

In [7]:
df['labels']=df['class'].map({0:"Offensive language detected",1:"Offensive language detected",2:"No Foul and Offensive speech"})
df[['tweet','labels']]

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Foul and Offensive speech
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive language detected
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive language detected
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive language detected
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive language detected
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,Offensive language detected
24779,"you've gone and broke the wrong heart baby, an...",No Foul and Offensive speech
24780,young buck wanna eat!!.. dat nigguh like I ain...,Offensive language detected
24781,youu got wild bitches tellin you lies,Offensive language detected


# Cleaning Dataset

In [8]:
def clean(txt):
    txt=str(txt).lower()
    txt=re.sub('\[.*?\]','',txt)
    txt=re.sub('https?://\S+|www\.\S+','',txt)
    txt=re.sub('<.*?>+','',txt)
    txt=re.sub('[%s]' % re.escape(string.punctuation),'',txt)
    txt=re.sub('\n','',txt)
    txt=re.sub('\w*\d\w*','',txt)
    txt=[word for word in txt.split(' ') if word not in stopword]
    txt=" ".join(txt)
    return txt
df['tweet']=df['tweet'].apply(clean)
df['tweet']

0         rt mayasolovely woman shouldnt complain clean...
1         rt  boy dats coldtyga dwn bad cuffin dat hoe ...
2         rt urkindofbrand dawg rt  ever fuck bitch sta...
3                 rt cganderson vivabased look like tranny
4         rt shenikaroberts shit hear might true might ...
                               ...                        
24778    yous muthafin lie   coreyemanuel right tl tras...
24779    youve gone broke wrong heart baby drove rednec...
24780    young buck wanna eat dat nigguh like aint fuck...
24781                    youu got wild bitches tellin lies
24782    ruffled  ntac eileen dahlia  beautiful color c...
Name: tweet, Length: 24783, dtype: object

# Prediction using Decison Tree Classifier

In [9]:
from sklearn.metrics import accuracy_score
x=np.array(df['tweet'])
y=np.array(df['labels'])

cv=CountVectorizer()
x=cv.fit_transform(x)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)
clf=DecisionTreeClassifier()
clf.fit(x_train,y_train)

y_pred = clf.predict(x_test)
accuarcy=accuracy_score(y_test,y_pred)

print(accuarcy*100)

93.66670742144517


# Foul Detection in Text

In [10]:
def check_text(test_data):
    df=cv.transform([test_data]).toarray()
    print(test_data)
    return clf.predict(df)

In [11]:
enter_text=input()
translated_text=translate(enter_text)
prediction=check_text(translated_text)
print(prediction)

stupid
stupid
['Offensive language detected']


# Foul Detection in Speech

In [12]:
import speech_recognition as sr
import pyaudio


def voiceanalysis():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Silence, please. Adjusting for ambient noise...")
        recognizer.adjust_for_ambient_noise(source, duration=2)  # Adjust based on ambient noise
        print("Speak now please:")

        try:
            audio = recognizer.listen(source)
            text = recognizer.recognize_google(audio)
            text = text.lower()
            print(text)
            return text
        except sr.UnknownValueError:
            return '0'
        except sr.RequestError as e:
            return '0'

        

# Video Analysis by comments and subtitles 

In [13]:
import requests
import json
from youtube_transcript_api import YouTubeTranscriptApi

# Replace YOUR_API_KEY with your actual API key and VIDEO_ID with the video ID
def commentretriever(vid):
    try:
        API_KEY = input("Enter your youtube api key to proceed:")
        VIDEO_ID = vid


        # Retrieving and Analysing Comments
        URL = f"https://www.googleapis.com/youtube/v3/commentThreads?key={API_KEY}&textFormat=plainText&part=snippet&videoId={VIDEO_ID}&maxResults=100"
        comments = []
        def fetch_comments(url):
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                for item in data['items']:

                    comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                    comments.append(comment)

                if 'nextPageToken' in data:
                    next_page_url = URL + "&pageToken=" + data['nextPageToken']
                    fetch_comments(next_page_url)

        fetch_comments(URL)
        return comments
    except:
        return []

     
def subtitleretriever(vid):

    # Retrieving and Analysing Subtitles
    try:
        subt=YouTubeTranscriptApi.get_transcript(vid)
        subtitles=[]
        for i in subt:
            subtitles.append(i['text'])
        return subtitles
    except:
        return []
        
        

# UI 

In [ ]:
from tkinter import *
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

root=Tk()
root.geometry('1165x900')
root['bg']='skyblue'
root.title('MLFD')
Label(root,text="MLFD",font="Arial 20 bold").pack()





# op=Text(root,width=50,height=1,wrap=WORD)
# op.place(x=550,y=150)


options=["Analyze Text","Analyze Speech","Video Context and Review Analysis"]
choose=ttk.Combobox(root,values=options,width=35)
choose.place(x=125,y=150)


    
def method1():
    Label(root,text="     Enter text       ",font="Arial 13 bold").place(x=665,y=100)
    inp=Entry(root,width=60)
    inp.place(x=550,y=150)
    def Testing1():
        text=inp.get()
        translated_text=translate(text)
        prediction=check_text(translated_text)
        Label(root,text="         Output            ",font="Arial 13 bold").place(x=640,y=100)
        op=Entry(root,width=60)
        op.place(x=550,y=150)
        op.insert(END,prediction[0])
    btn=Button(root,text='Test',font="Arial 13 bold",command=Testing1).place(x=700,y=200)
    

def method2():
    def listening():
        listbox = Listbox()
        listbox.pack(padx=10, pady=10, expand=True)
        listbox.place(x=125, y=350, width=500, height=250)
        text=voiceanalysis()
        if text=='0':
            listbox.insert(END,"Can't understand the language")
        else:
            translated_text=translate(text)
            allwords=list(translated_text.split())
            allfoulwords=[]
            for i in allwords:
                prediction=check_text(i)
                if prediction[0]=='Offensive language detected':
                    allfoulwords.append(i)
            foul_word_percent=(len(allfoulwords)/len(allwords))*100
            fig1 = Figure(figsize=(2, 2), dpi=100)
            plot1 = fig1.add_subplot(1, 1, 1)
            labels1 = 'Hate', 'Clean'
            colors1 = ['#ff9999', '#66b3ff']
            sizes1 = [foul_word_percent,100-foul_word_percent]
            plot1.pie(sizes1,colors=colors1, labels=labels1, autopct='%1.1f%%', startangle=90)
            plot1.axis('equal')
            canvas = FigureCanvasTkAgg(fig1)  
            canvas.draw()
            canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
            canvas.get_tk_widget().place(x=755, y=350, width=250, height=250)
            Label(root,text="speech analysis",font="Arial 13 bold").place(x=815,y=615)
            listbox.insert(END,"Speech is :")
            listbox.insert(END," ")
            listbox.insert(END,text)
            Label(root,text="         Output            ",font="Arial 13 bold").place(x=640,y=100)
            op=Entry(root,width=60)
            op.place(x=550,y=150)
            if len(allfoulwords):
                op.insert(END,"Foul and Offensive Speech")
            else:
                op.insert(END,"No Foul in Speech")
    listbox = Listbox()
    listbox.pack(padx=10, pady=10, expand=True)
    listbox.place(x=125, y=250, width=250, height=40)
    listbox.insert(END,"Silence, please. Adjusting for ambient noise...")
    listbox.insert(END,"Click the button and speak please...\n\n")
    btn=Button(root,text='Speak',font="Arial 13 bold",command=listening).place(x=400,y=255)
    

        
        
def findid(url):
    l=len(url)
    s=""
    for i in range(1,l-1):
        if url[i]=='/' and url[i+1]!='/' and url[i-1]!='/':
                for j in range(i+1,l):
                    if(url[j]=='?'):
                        return s
                    s+=url[j]
                
def method3():
    Label(root,text="Enter url of video",font="Arial 13 bold").place(x=665,y=100)
    inp=Entry(root,width=60)
    inp.place(x=550,y=150)
    def Testing2():
        text=inp.get()
        vid=findid(text)
        
        allcomments=commentretriever(vid)
        allsubtitles=subtitleretriever(vid)
        allfoulcomments=[]
        allfoulsubtitles=[]
        
        
        if len(allcomments):
            for i in allcomments:
                translated_text=translate(i)
                prediction=check_text(translated_text)
                if(prediction[0]=='Offensive language detected'):
                    allfoulcomments.append(i)
            foul_comment_percent=(len(allfoulcomments)/len(allcomments))*100
        else:
            foul_comment_percent=0
            
        if len(allsubtitles):
            for i in allsubtitles:
                translated_text=translate(i)
                prediction=check_text(translated_text)
                if(prediction[0]=='Offensive language detected'):
                    allfoulsubtitles.append(i)
            foul_subtitle_percent=(len(allfoulsubtitles)/len(allsubtitles))*100
        else:
            foul_subtitle_percent=0
        
        
        
        Label(root,text="         Output            ",font="Arial 13 bold").place(x=640,y=100)
        op=Entry(root,width=60)
        op.place(x=550,y=150)
        if(foul_comment_percent+foul_subtitle_percent>40):
            op.insert(END,'Video is not prefered to watch based on vulgarity analysis')
        else:
            op.insert(END,'Video is safe to watch')
        
        Label(root,text=" allcomments ",font="Arial 13 bold").place(x=70,y=275)
        listbox1 = Listbox()
        listbox1.pack(padx=10, pady=10, expand=True)
        listbox1.place(x=10, y=300, width=250, height=250)
        for item in allcomments:
            listbox1.insert(END, item)
        
        Label(root,text="foulcomments",font="Arial 13 bold").place(x=70,y=575)
        listbox2 = Listbox()
        listbox2.pack(padx=10, pady=10, expand=True)
        listbox2.place(x=10, y=600, width=250, height=250)
        for item in allfoulcomments:
            listbox2.insert(END, item)
        
        Label(root,text="allsubtitles",font="Arial 13 bold").place(x=675,y=275)
        listbox3 = Listbox()
        listbox3.pack(padx=10, pady=10, expand=True)
        listbox3.place(x=605, y=300, width=250, height=250)
        for item in allsubtitles:
            listbox3.insert(END, item)
        
        Label(root,text="foulsubtitles",font="Arial 13 bold").place(x=675,y=575)
        listbox4 = Listbox()
        listbox4.pack(padx=10, pady=10, expand=True)
        listbox4.place(x=605, y=600, width=250, height=250)
        for item in allfoulsubtitles:
            listbox4.insert(END, item)
            
        
        fig1 = Figure(figsize=(2, 2), dpi=100)
        plot1 = fig1.add_subplot(1, 1, 1)
        labels1 = 'Hate', 'Clean'
        colors1 = ['#ff9999', '#66b3ff']
        sizes1 = [foul_comment_percent,100-foul_comment_percent]
        plot1.pie(sizes1,colors=colors1, labels=labels1, autopct='%1.1f%%', startangle=90)
        plot1.axis('equal')
        canvas = FigureCanvasTkAgg(fig1)  
        canvas.draw()
        canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
        canvas.get_tk_widget().place(x=305, y=300, width=250, height=250)
        Label(root,text="comments analysis",font="Arial 13 bold").place(x=355,y=565)
        
        fig2 = Figure(figsize=(2, 2), dpi=100)
        plot2 = fig2.add_subplot(1, 1, 1)
        labels2 = 'Hate', 'Clean'
        colors2 = ['#ff9999', '#66b3ff']
        sizes2 = [foul_subtitle_percent,100-foul_subtitle_percent]
        plot2.pie(sizes2,colors=colors2,labels=labels2, autopct='%1.1f%%', startangle=90)
        plot2.axis('equal')
        canvas = FigureCanvasTkAgg(fig2)  
        canvas.draw()
        canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
        canvas.get_tk_widget().place(x=905, y=300, width=250, height=250)
        Label(root,text="",font="Arial 13 bold").place(x=70,y=275)
        Label(root,text="subtitles analysis",font="Arial 13 bold").place(x=965,y=565)
        
        
    btn=Button(root,text='Test',font="Arial 13 bold",command=Testing2).place(x=700,y=200)
    
def Proceed():
    selected=choose.get()
    if(selected==options[0]):
        method1()
    if(selected==options[1]):
        method2()
    if(selected==options[2]):
        method3()
        
btn=Button(root,text='Proceed',font="Arial 13 bold",command=Proceed).place(x=400,y=145)
    
    
root.mainloop()